In [442]:
from scipy.io import FortranFile
import numpy as np
from math import pi
from itertools import compress
from astropy.io import ascii
import argparse
import os
import numba

In [443]:
"""
Histogram Cherenkov and fluorescence photon bunches on the ground.
In both ways, 2D histogram or lateral photon density profile.
"""
pointing = 0.0
fov_diameter = 10

In [444]:
def histogram(photon_bunches, x_area, y_area, pointing_ang, nshower):
    """
    :param photon_bunches: array containing only photon bunches
    sub-blocks. The information contained is:
    [N_phot, x, y, uemis, vemis, t_emis, height of emission]
    :param x_area: x dimension of detection area (in cm)
    :param y_area: x dimension of detection area (in cm)
    :param pointing_ang: telescope pointing angle
                         with respect to vertical direction
    :param nshower: number of simulated showers
    :return: photons histogrammed radially/along x or y-axis
    """
    fov = np.cos(0.5 * fov_diameter * pi / 180)  # FoV constraint radially
    # Histogram along x-axis
    if x_area > y_area:
        weighted_pht = np.abs(photon_bunches[:, 0]) / (binsize**2 * nshower)
        h, edges = np.histogram(1e-2 * photon_bunches[:, 1],
                                bins=distances,
                                weights=weighted_pht,
                                range=[0., maxlen]
                                )
        wemis = np.sqrt(1-photon_bunches[:, 3]**2 - photon_bunches[:, 4]**2)
        # Pointing telescope along the pointing angle direction
        # set as input argument
        wemis = (wemis * np.cos(-pointing_ang * pi / 180)
                 + photon_bunches[:, 3] * np.sin(-pointing_ang * pi / 180))
        bunches_fov = photon_bunches[wemis >= fov]
        weighted_pht_fov = weighted_pht[wemis >= fov]
        h_fov, edges = np.histogram(1e-2 * bunches_fov[:, 1],
                                    bins=distances,
                                    weights=weighted_pht_fov,
                                    range=[0., maxlen]
                                    )
    elif x_area < y_area:
        weighted_pht = np.abs(photon_bunches[:, 0]) / (binsize**2 * nshower)
        h, edges = np.histogram(1e-2 * photon_bunches[:, 2],
                                bins=distances,
                                weights=weighted_pht,
                                range=[0., maxlen]
                                )
        wemis = np.sqrt(1 - photon_bunches[:, 3]**2 - photon_bunches[:, 4]**2)
        wemis = (wemis * np.cos(-pointing_ang * pi / 180)
                 + photon_bunches[:, 3] * np.sin(-pointing_ang * pi / 180))
        bunches_fov = photon_bunches[wemis >= fov]
        weighted_pht_fov = weighted_pht[wemis >= fov]
        h_fov, edges = np.histogram(1e-2 * bunches_fov[:, 2],
                                    bins=distances,
                                    weights=weighted_pht_fov,
                                    range=[0., maxlen]
                                    )

    # Histogram radially
    else:
        type_of_hist = 'r'
        radius = np.linspace(0, maxlen, breaks)
        mids = ((radius[1] - radius[0]) / 2) + radius
        mids = mids[0:numbins]
        ring = pi * ((radius[1:]) ** 2 - (radius[0:numbins]) ** 2)
        r = np.sqrt((1e-2 * photon_bunches[:, 1])**2
                    + (1e-2 * photon_bunches[:, 2])**2)
        photon_bunches = photon_bunches[r < maxlen]
        r = r[r < maxlen]
        ring2 = ring[(r / (radius[1] - radius[0])).astype(int)]
        weighted_pht = np.abs(photon_bunches[:, 0]) / (ring2 * nshower)

        # Store into the histogram
        h, edges = np.histogram(r,
                                bins=radius,
                                weights=weighted_pht,
                                range=[0., maxlen]
                                )
        wemis = np.sqrt(1 - photon_bunches[:, 3]**2 - photon_bunches[:, 4]**2)
        wemis = (wemis * np.cos(-pointing_ang * pi / 180)
                 + photon_bunches[:, 3] * np.sin(-pointing_ang * pi / 180))
        bunches_fov = photon_bunches[wemis >= fov]
        r_fov = r[wemis >= fov]
        ring2_fov = ring[(r_fov / (radius[1] - radius[0])).astype(int)]
        weighted_pht_fov = np.abs(bunches_fov[:, 0]) / (ring2_fov * nshower)
        h_fov, edges = np.histogram(r_fov,
                                    bins=radius,
                                    weights=weighted_pht_fov,
                                    range=[0., maxlen]
                                    )
    return h, h_fov

In [445]:
dcard = {}
with open("all", 'r') as f:
    """
    Read input variables from DATACARD
    """
    read_data = f.read()
    raw_datacard = ascii.read(read_data, format='fixed_width_no_header',
                              delimiter=' ', col_starts=(0, 8, 39))

for keyword, value, description in raw_datacard:
    while True:
        try:
            dcard[str(keyword)] = [float(x) for x in value.split()]
        except ValueError:
            dcard[str(keyword)] = [str(x) for x in value.split()]
        except AttributeError:
            print('DataCard successfully loaded')
        break

# Redefining some of the fields
dcard['THETAP'] = dcard['THETAP'][0]
dcard['XCERARY'] = dcard['CERARY'][4]
dcard['YCERARY'] = dcard['CERARY'][5]
dcard['ERANGE'] = dcard['ERANGE'][0]
dcard['SEED'] = dcard['SEED'][0]
dcard['NSHOW'] = dcard['NSHOW'][0]
dcard['OBSLEV'] = dcard['OBSLEV'][0]
dcard['CERSIZ'] = dcard['CERSIZ'][0]
dcard['FLSIZE'] = dcard['FLSIZE'][0]
dcard['PRMPAR'] = dcard['PRMPAR'][0]
dcard['ATMOD'] = dcard['ATMOD'][0]


# Telescope pointing
if pointing == dcard['THETAP']:
    pointing_angle = dcard['THETAP']
    pointing = 'onaxis'
    off_axis = 0.
else:
    pointing_angle = pointing
    pointing = 'offaxis'
    off_axis = pointing_angle - dcard['THETAP']

DataCard successfully loaded


In [446]:
# Open CORSIKA output binary file
file = FortranFile("CER000001", 'r')

# Histograms definition
binsize = 10  # meters
maxlen = 0.5e-2 * max(dcard['XCERARY'], dcard['YCERARY'])
numbins = int(maxlen / binsize)
breaks = numbins + 1
distances = np.linspace(0, maxlen, breaks)
distances2D = np.linspace(-maxlen, maxlen, 2*breaks-1)
mids = ((distances[1] - distances[0]) / 2) + distances
mids = mids[0:numbins]
hist_c = np.zeros((2, numbins))
hist_f = np.zeros((2, numbins))


if dcard['XCERARY'] > dcard['YCERARY']:
    type_of_hist = 'x'

elif dcard['XCERARY'] < dcard['YCERARY']:
    type_of_hist = 'y'

else:
    type_of_hist = 'r'
    radius = np.linspace(0, maxlen, breaks)
    mids = ((radius[1] - radius[0]) / 2) + radius
    mids = mids[0:numbins]
    ring = pi * ((radius[1:]) ** 2 - (radius[0:numbins]) ** 2)

In [447]:
def _read(file):
    raw_block = file.read_reals(dtype=np.float32)
    return raw_block


@numba.jit(nopython=True)
def _organize(raw_data):
    block = raw_data.reshape(-1, 7)
    return block


@numba.jit()
def _get_block_indices(block, indices, indices_bool):
    # Select only sub-blocks of bunches
    for i, arrays in enumerate(block):
        indices[i] = arrays[0][0]
        if np.abs(arrays[0][0]) < 3000:
            indices_bool[i] = True

def _keep_photon_bunches(bunches, data, indices_bool):
    # Keep only photon bunches
    bunches = np.vstack([bunches, np.vstack(compress(data, indices_bool))])
    return bunches

@numba.jit()
def _drop_all_zeros(bunches):
    bunches_clean = bunches[~np.all(bunches == 0, axis=1)]
    return bunches_clean

@numba.jit()
def _sum_histograms(hist_c, hist_f, h_c, h_f):
    hist_c = hist_c + h_c
    hist_f = hist_f + h_f
    return hist_c, hist_f


def photon_block():
    
    # Control counter and empty photon bunches array definition:
    count = 0
    bunches = np.array([]).reshape(0, 7)
    hist_c = np.zeros((2, numbins))
    hist_f = np.zeros((2, numbins))
    indices_bool = np.zeros(21, dtype = bool)
    indices = np.empty(21, dtype = float)
       
    while True:
        count = count + 1
        
        raw_block = _read(file)
        
        block_columns = _organize(raw_block)
        
        # Sort data in 21 sub-blocks of 39 lines each
        blocks = np.split(block_columns, 21)
        
        _get_block_indices(blocks, indices, indices_bool)
        
        bunches = _keep_photon_bunches(bunches, blocks, indices_bool)
        
        # Drop those lines containing only zeros
        bunches = _drop_all_zeros(bunches)

        if count == 10:

            h_c = histogram(bunches[bunches[:, 0] > 0],
                            dcard['XCERARY'], dcard['YCERARY'],
                            pointing_angle, dcard['NSHOW'])

            h_f = histogram(bunches[bunches[:, 0] < 0],
                            dcard['XCERARY'], dcard['YCERARY'],
                            pointing_angle, dcard['NSHOW'])
            
            hist_photons = _sum_histograms(hist_c, hist_f, h_c, h_f)
            
            # Reset
            count = 0
            bunches = np.array([]).reshape(0, 7)

        if any(3300 < i < 3303. for i in indices):  # RUN END subblock
            if count < 10:
                h_c = histogram(bunches[bunches[:, 0] > 0],
                                dcard['XCERARY'], dcard['YCERARY'],
                                pointing_angle, dcard['NSHOW'])
                
                h_f = histogram(bunches[bunches[:, 0] < 0],
                                dcard['XCERARY'], dcard['YCERARY'],
                                pointing_angle, dcard['NSHOW'])
                
                hist_photons = _sum_histograms(hist_c, hist_f, h_c, h_f)
                
            return hist_photons    
            file.close()
            break
        

In [448]:
c, f = photon_block()
%timeit photon_block

/home/danilo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in sqrt
/home/danilo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in greater_equal
/home/danilo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in greater_equal


10000000 loops, best of 3: 21.3 ns per loop


In [434]:
np.savetxt('%iGeV_%ish_%ideg_%i%s_hist_%s_%iFoV.dat' % (dcard['ERANGE'],
                                                        dcard['NSHOW'],
                                                        dcard['THETAP'],
                                                        abs(off_axis),
                                                        pointing,
                                                        type_of_hist,
                                                        fov_diameter),
           np.transpose([mids, c[0], c[1],  # Cherenkov
                         f[0], f[1]]),  # fluorescence
           newline='\n',
           fmt="%7.2f %1.6e %1.6e %1.6e %1.6e",
           header=(' Number of showers: %i \n Energy primary [GeV]: %i \n'
                   ' ID primary particle: %s \n'
                   ' Seed for Cherenkov emission: %i \n'
                   % (dcard['NSHOW'], dcard['ERANGE'], dcard['PRMPAR'], dcard['SEED'])
                   +
                   ' Theta primary particle [deg]: %i \n'
                   ' Observation level [cm]: %i \n'
                   ' Atm. model: %i \n'
                   ' Diameter FoV [deg]: %i \n'
                   % (dcard['THETAP'], dcard['OBSLEV'], dcard['ATMOD'], fov_diameter)
                   +
                   ' Cherenkov bunch size: %i \n Fluor bunch size: %i \n'
                   % (dcard['CERSIZ'], dcard['FLSIZE'])
                   +
                   ' Distance to core [m] |'
                   ' Ph. density [1/m2] C (all) | C (FoV) |'
                   ' F (all) | F (FoV)'
                   )
           )